In [34]:
import pandas as pd
import os
import re

In [35]:
folders = [f for f in os.listdir('..') if '__profile_run' in f]
folders

['268435456_table-words__512_updates-per-core__32_cores__16_concurrency__profile_run',
 '268435456_table-words__512_updates-per-core__64_cores__1_concurrency__profile_run',
 '268435456_table-words__512_updates-per-core__1_cores__4_concurrency__profile_run',
 '268435456_table-words__512_updates-per-core__1_cores__8_concurrency__profile_run',
 '268435456_table-words__512_updates-per-core__2_cores__8_concurrency__profile_run',
 '268435456_table-words__512_updates-per-core__2_cores__1_concurrency__profile_run',
 '268435456_table-words__512_updates-per-core__2_cores__16_concurrency__profile_run',
 '268435456_table-words__512_updates-per-core__4_cores__4_concurrency__profile_run',
 '268435456_table-words__512_updates-per-core__8_cores__1_concurrency__profile_run',
 '268435456_table-words__512_updates-per-core__8_cores__8_concurrency__profile_run',
 '268435456_table-words__512_updates-per-core__16_cores__1_concurrency__profile_run',
 '268435456_table-words__512_updates-per-core__16_cores__8_c

In [36]:
def vcache_name(x,y):
    return "replicant_tb_top.testbench.DUT.py[0].podrow.px[0].pod.north_vc_x[0].north_vc_row.vc_y[%d].vc_x[%d].vc.cache.vcache_prof" % (y,x)

def vcache(vcaches_data):
    # get the min and max time
    min_time = vcaches_data['time'].min()
    max_time = vcaches_data['time'].max()
    # get the min and max global ctr
    min_global_ctr = vcaches_data['global_ctr'].min()
    max_global_ctr = vcaches_data['global_ctr'].max()
    # just keep the min and max time
    vcaches_data = vcaches_data[(vcaches_data['time']==min_time)|(vcaches_data['time']==max_time)]
    summary = vcaches_data.groupby(['time','tag','global_ctr','table_words','updates_per_core','cores','concurrency']).sum().diff().dropna()
    # mark time
    summary['time_start'] = min_time
    summary['time_end'] = max_time
    summary['time_elapsed'] = max_time-min_time
    # mark cycles
    summary['global_ctr_start'] = min_global_ctr
    summary['global_ctr_end'] = max_global_ctr
    summary['global_ctr_elapsed'] = max_global_ctr-min_global_ctr
    return summary
    
def vcore(vcores_data):
    full_summary = pd.DataFrame()
    for x in range(16):
        for y in range(8):
            # filter only this vcore
            vcore_data = vcores_data[(vcores_data['x']==x)&(vcores_data['y']==y)]
            if vcore_data.empty: continue
            # get the min and max time
            min_time = vcore_data['time'].min()
            max_time = vcore_data['time'].max()
            # get the min and max global ctr
            min_global_ctr = vcore_data['global_ctr'].min()
            max_global_ctr = vcore_data['global_ctr'].max()
            # just keep the min and max time    
            vcore_data = vcore_data[(vcore_data['time']==min_time)|(vcore_data['time']==max_time)]
            summary = vcore_data.groupby(['time','tag','x','y','global_ctr','table_words','updates_per_core','cores','concurrency']).sum().diff().dropna()
            # mark time
            summary['time_start'] = min_time
            summary['time_end'] = max_time
            summary['time_elapsed']=max_time-min_time
            # mark cycles
            summary['global_ctr_start'] = min_global_ctr
            summary['global_ctr_end'] = max_global_ctr
            summary['global_ctr_elapsed'] = max_global_ctr-min_global_ctr
            # append
            full_summary = full_summary.append(summary)
    # return the full summary
    return full_summary

def folder(fldr):
    # parse the folder name to get experiment parameters
    m = re.match(r'^(\d+)_table-words__(\d+)_updates-per-core__(\d+)_cores__(\d+)_concurrency__profile_run$', fldr)
    table_words = int(m.group(1))
    updates_per_core = int(m.group(2))
    cores = int(m.group(3))
    concurrency = int(m.group(4))    
    # read data from experiment
    vcaches_data_file = '../' + fldr + '/vcache_stats.csv'
    vcores_data_file  = '../' + fldr + '/vanilla_stats.csv'
    vcaches_data = pd.read_csv(vcaches_data_file) 
    vcores_data = pd.read_csv(vcores_data_file)
    # post process vcache data    
    vcaches_data['table_words']=table_words
    vcaches_data['updates_per_core']=updates_per_core
    vcaches_data['cores']=cores
    vcaches_data['concurrency']=concurrency
    vcache_data = vcache(vcaches_data)
    # post process vcore data
    vcores_data['table_words']=table_words
    vcores_data['updates_per_core']=updates_per_core
    vcores_data['cores']=cores
    vcores_data['concurrency']=concurrency
    vcore_data = vcore(vcores_data)
    return (vcache_data, vcore_data)

folder(folders[0])

(                                                                                instr_ld  \
 time      tag        global_ctr table_words updates_per_core cores concurrency             
 464159000 3444310480 464143     268435456   512              32    16              242.0   
 
                                                                                 instr_ld_ld  \
 time      tag        global_ctr table_words updates_per_core cores concurrency                
 464159000 3444310480 464143     268435456   512              32    16                   0.0   
 
                                                                                 instr_ld_ldu  \
 time      tag        global_ctr table_words updates_per_core cores concurrency                 
 464159000 3444310480 464143     268435456   512              32    16                    0.0   
 
                                                                                 instr_ld_lw  \
 time      tag        global_ctr table_w

In [37]:
vcache_summary_data = pd.DataFrame()
vcore_summary_data = pd.DataFrame()
for fldr in folders:
    cache,core = folder(fldr)
    vcache_summary_data = vcache_summary_data.append(cache)
    vcore_summary_data = vcore_summary_data.append(core)
vcore_summary_data

,,,,,,,,,pc_r,pc_n,cycle,instr_total,instr_fadd,instr_fsub,instr_fmul,instr_fsgnj,instr_fsgnjn,instr_fsgnjx,...,stall_fcsr,stall_remote_ld_wb,stall_ifetch_wait,stall_remote_flw_wb,time_start,time_end,time_elapsed,global_ctr_start,global_ctr_end,global_ctr_elapsed
time,tag,x,y,global_ctr,table_words,updates_per_core,cores,concurrency,,,,,,,,,,,,,,,,,,,,,
422293000,3359637504,0,0,422277,268435456,512,32,16,197.0,197.0,164899.0,5591.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,257394000,422293000,164899000,257378,422277,164899
443475000,3376414736,0,1,443459,268435456,512,32,16,197.0,197.0,185551.0,5591.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,257924000,443475000,185551000,257908,443459,185551
432446000,3393191968,0,2,432430,268435456,512,32,16,197.0,197.0,174203.0,5591.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,258243000,432446000,174203000,258227,432430,174203
431176000,3409969200,0,3,431160,268435456,512,32,16,197.0,197.0,172576.0,5591.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,258600000,431176000,172576000,258584,431160,172576
448532000,3426746432,0,4,448516,268435456,512,32,16,198.0,198.0,189529.0,5591.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,259003000,448532000,189529000,258987,448516,189529
440903000,3443523664,0,5,440887,268435456,512,32,16,198.0,198.0,181379.0,5592.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,259524000,440903000,181379000,259508,440887,181379
437935000,3460300896,0,6,437919,268435456,512,32,16,198.0,198.0,177938.0,5592.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,259997000,437935000,177938000,259981,437919,177938
447582000,3477078128,0,7,447566,268435456,512,32,16,198.0,198.0,187073.0,5592.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,260509000,447582000,187073000,260493,447566,187073
434946000,3359899776,1,0,434930,268435456,512,32,16,197.0,197.0,174064.0,5591.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,260882000,434946000,174064000,260866,434930,174064


In [38]:
vcache_summary_data.to_csv('vcache_summary.txt')

In [39]:
vcore_summary_data.to_csv('vcore_summary.txt')